In [206]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import re
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from scipy.stats import randint
import io
import pickle
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

In [120]:
column_names_ratings = ['UserID', 'MovieID', 'Rating', 'Timestamp']
column_names_user = ['UserID', 'Gender', 'Age', 'Occupation', 'Zip-code']
column_names_movies = ['MovieID', 'Title', 'Genres']
ratings_df = pd.read_csv('ratings.dat', delimiter='::',names=column_names_ratings)
user_df = pd.read_csv('users.dat', delimiter='::',names=column_names_user)
movies_df=pd.read_csv('movies.dat', delimiter='::',names=column_names_movies,encoding='latin1')




C:\Users\20109\AppData\Local\Temp\ipykernel_20340\2301806548.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ratings_df = pd.read_csv('ratings.dat', delimiter='::',names=column_names_ratings)
C:\Users\20109\AppData\Local\Temp\ipykernel_20340\2301806548.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  user_df = pd.read_csv('users.dat', delimiter='::',names=column_names_user)
C:\Users\20109\AppData\Local\Temp\ipykernel_20340\2301806548.py:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are i

In [121]:
ratings_df.head()

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [122]:
user_df.head()

,UserID,Gender,Age,Occupation,Zip-code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [123]:
movies_df.head()

,MovieID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [124]:
movies_df.head()

,MovieID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [125]:
nan_values = movies_df['Genres'].isna().sum()
print(f"Number of NaN values in 'Genres' column: {nan_values}")

Number of NaN values in 'Genres' column: 0


In [126]:
movies_df['Genres'] = movies_df['Genres'].str.split('|')

# Initialize MultiLabelBinarizer
mlb = MultiLabelBinarizer()

# Perform one-hot encoding
encoded_genres = pd.DataFrame(mlb.fit_transform(movies_df['Genres']), columns=mlb.classes_, index=movies_df.index)

# Concatenate with the original DataFrame
encoded_data = pd.concat([movies_df, encoded_genres], axis=1)

# Drop the original 'genres' column
encoded_data.drop('Genres', axis=1, inplace=True)

In [127]:
encoded_data

,MovieID,Title,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji (1995),0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men (1995),0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
3,4,Waiting to Exhale (1995),0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,5,Father of the Bride Part II (1995),0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3878,3948,Meet the Parents (2000),0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3879,3949,Requiem for a Dream (2000),0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3880,3950,Tigerland (2000),0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3881,3951,Two Family House (2000),0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [128]:
encoded_data['Year'] = encoded_data['Title'].apply(lambda x: re.search(r'\((\d{4})\)', x).group(1))
encoded_data['Title'] = encoded_data['Title'].apply(lambda x: re.sub(r'\s*\(\d{4}\)', '', x))

In [169]:
encoded_data.head()


,MovieID,Title,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,Year
0,1,Toy Story,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1995
1,2,Jumanji,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1995
2,3,Grumpier Old Men,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1995
3,4,Waiting to Exhale,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1995
4,5,Father of the Bride Part II,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1995


In [130]:
movies_new=encoded_data.drop(columns=['Title'])

,MovieID,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,Year
0,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1995
1,2,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1995
2,3,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1995
3,4,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1995
4,5,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1995


In [170]:
encoded_users_df = pd.get_dummies(user_df, columns=[ 'Gender'], prefix=['Gender'], dtype='int')

In [171]:
encoded_users_df.head()

,UserID,Age,Occupation,Zip-code,Gender_F,Gender_M
0,1,1,10,48067,1,0
1,2,56,16,70072,0,1
2,3,25,15,55117,0,1
3,4,45,7,02460,0,1
4,5,25,20,55455,0,1


In [133]:
encoded_users_df=encoded_users_df.drop(columns=['Zip-code'])

In [134]:
pd.set_option('display.max_columns',None)
encoded_users_df.head()

,UserID,Age,Occupation,Gender_F,Gender_M
0,1,1,10,1,0
1,2,56,16,0,1
2,3,25,15,0,1
3,4,45,7,0,1
4,5,25,20,0,1


In [135]:
merged_data = pd.merge(ratings_df, movies_new, on='MovieID', how='left')
final_merged_data = pd.merge(merged_data, encoded_users_df, on='UserID', how='left')

In [136]:
final_merged_data.head(30)

,UserID,MovieID,Rating,Timestamp,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,Year,Age,Occupation,Gender_F,Gender_M
0,1,1193,5,978300760,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1975,1,10,1,0
1,1,661,3,978302109,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1996,1,10,1,0
2,1,914,3,978301968,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1964,1,10,1,0
3,1,3408,4,978300275,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2000,1,10,1,0
4,1,2355,5,978824291,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1998,1,10,1,0
5,1,1197,3,978302268,1,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1987,1,10,1,0
6,1,1287,5,978302039,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1959,1,10,1,0
7,1,2804,5,978300719,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1983,1,10,1,0
8,1,594,4,978302268,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1937,1,10,1,0
9,1,919,4,978301368,0,1,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1939,1,10,1,0


In [137]:
data=final_merged_data.drop(columns=['Timestamp'])

In [70]:
data.head(20)

,UserID,MovieID,Rating,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,Year,Age,Occupation,Gender_F,Gender_M
0,1,1193,5,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1975,1,10,1,0
1,1,661,3,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1996,1,10,1,0
2,1,914,3,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1964,1,10,1,0
3,1,3408,4,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2000,1,10,1,0
4,1,2355,5,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1998,1,10,1,0
5,1,1197,3,1,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1987,1,10,1,0
6,1,1287,5,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1959,1,10,1,0
7,1,2804,5,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1983,1,10,1,0
8,1,594,4,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1937,1,10,1,0
9,1,919,4,0,1,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1939,1,10,1,0


In [71]:
data.shape

(1000209, 26)

In [176]:
genre_columns = final_merged_data.columns[3:22]  # Assuming genre columns start from index 3
rating_column = 'Rating'

selected_columns = ['UserID','MovieID', rating_column] + list(genre_columns)
User_genre_rating=final_merged_data[selected_columns]


In [177]:
User_genre_rating.head()

,UserID,MovieID,Rating,Timestamp,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,1193,5,978300760,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,1,661,3,978302109,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2,1,914,3,978301968,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0
3,1,3408,4,978300275,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4,1,2355,5,978824291,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [178]:
User_genre_rating[genre_columns] = User_genre_rating[genre_columns].multiply(User_genre_rating[rating_column], axis=0)


C:\Users\20109\AppData\Local\Temp\ipykernel_20340\554428280.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  User_genre_rating[genre_columns] = User_genre_rating[genre_columns].multiply(User_genre_rating[rating_column], axis=0)


In [179]:
User_genre_rating.head()

,UserID,MovieID,Rating,Timestamp,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,1193,5,4891503800,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,0,0
1,1,661,3,2934906327,0,0,3,3,0,0,0,0,0,0,0,3,0,0,0,0,0,0
2,1,914,3,2934905904,0,0,0,0,0,0,0,0,0,0,0,3,0,3,0,0,0,0
3,1,3408,4,3913201100,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0
4,1,2355,5,4894121455,0,0,5,5,5,0,0,0,0,0,0,0,0,0,0,0,0,0


In [189]:
# User_genre_rating=User_genre_rating.drop(columns='Timestamp')

In [190]:
User_genre_rating.head()


,UserID,MovieID,Rating,Timestamp,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,1193,5,4891503800,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,0,0
1,1,661,3,2934906327,0,0,3,3,0,0,0,0,0,0,0,3,0,0,0,0,0,0
2,1,914,3,2934905904,0,0,0,0,0,0,0,0,0,0,0,3,0,3,0,0,0,0
3,1,3408,4,3913201100,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0
4,1,2355,5,4894121455,0,0,5,5,5,0,0,0,0,0,0,0,0,0,0,0,0,0


In [191]:
genre_mean_df = User_genre_rating.groupby('UserID')[genre_columns].mean().reset_index()
genre_mean_df.rename(columns=lambda x: f'user_{x}', inplace=True)


In [192]:

genre_mean_df.rename(columns={ 'user_UserID':'UserID'},inplace=True)

In [193]:
genre_mean_df.head()

,UserID,user_Timestamp,user_Action,user_Adventure,user_Animation,user_Children's,user_Comedy,user_Crime,user_Documentary,user_Drama,user_Fantasy,user_Film-Noir,user_Horror,user_Musical,user_Mystery,user_Romance,user_Sci-Fi,user_Thriller,user_War,user_Western
0,1,4.098334e+09,0.396226,0.377358,1.396226,1.603774,1.094340,0.150943,0.000000,1.754717,0.226415,0.000000,0.000000,1.132075,0.000000,0.415094,0.245283,0.207547,0.188679,0.000000
1,2,3.632600e+09,1.519380,0.550388,0.000000,0.000000,0.689922,0.333333,0.000000,2.387597,0.023256,0.031008,0.046512,0.000000,0.077519,0.689922,0.472868,0.837209,0.434109,0.100775
2,3,3.817280e+09,1.784314,1.960784,0.235294,0.235294,2.215686,0.000000,0.000000,0.627451,0.176471,0.000000,0.156863,0.078431,0.058824,0.372549,0.450980,0.372549,0.156863,0.549020
3,4,4.099519e+09,3.761905,1.095238,0.000000,0.190476,0.000000,0.238095,0.000000,1.190476,0.428571,0.000000,0.619048,0.000000,0.000000,0.380952,1.523810,0.666667,0.476190,0.428571
4,5,3.078012e+09,0.409091,0.136364,0.080808,0.116162,0.964646,0.348485,0.111111,1.626263,0.000000,0.060606,0.141414,0.050505,0.126263,0.469697,0.232323,0.560606,0.106061,0.020202


In [194]:
genre_mean_df.head(20)


,UserID,user_Timestamp,user_Action,user_Adventure,user_Animation,user_Children's,user_Comedy,user_Crime,user_Documentary,user_Drama,user_Fantasy,user_Film-Noir,user_Horror,user_Musical,user_Mystery,user_Romance,user_Sci-Fi,user_Thriller,user_War,user_Western
0,1,4.098334e+09,0.396226,0.377358,1.396226,1.603774,1.094340,0.150943,0.000000,1.754717,0.226415,0.000000,0.000000,1.132075,0.000000,0.415094,0.245283,0.207547,0.188679,0.000000
1,2,3.632600e+09,1.519380,0.550388,0.000000,0.000000,0.689922,0.333333,0.000000,2.387597,0.023256,0.031008,0.046512,0.000000,0.077519,0.689922,0.472868,0.837209,0.434109,0.100775
2,3,3.817280e+09,1.784314,1.960784,0.235294,0.235294,2.215686,0.000000,0.000000,0.627451,0.176471,0.000000,0.156863,0.078431,0.058824,0.372549,0.450980,0.372549,0.156863,0.549020
3,4,4.099519e+09,3.761905,1.095238,0.000000,0.190476,0.000000,0.238095,0.000000,1.190476,0.428571,0.000000,0.619048,0.000000,0.000000,0.380952,1.523810,0.666667,0.476190,0.428571
4,5,3.078012e+09,0.409091,0.136364,0.080808,0.116162,0.964646,0.348485,0.111111,1.626263,0.000000,0.060606,0.141414,0.050505,0.126263,0.469697,0.232323,0.560606,0.106061,0.020202
5,6,3.816503e+09,0.619718,0.422535,0.535211,0.873239,1.690141,0.084507,0.000000,1.154930,0.084507,0.000000,0.000000,1.154930,0.000000,1.915493,0.098592,0.154930,0.281690,0.408451
6,7,4.228499e+09,4.032258,1.193548,0.000000,0.000000,0.419355,0.258065,0.000000,1.032258,0.000000,0.000000,0.161290,0.000000,0.129032,0.387097,1.387097,2.225806,0.612903,0.000000
7,8,3.800320e+09,1.043165,0.251799,0.122302,0.064748,0.503597,0.374101,0.000000,2.964029,0.000000,0.000000,0.035971,0.028777,0.100719,0.877698,0.417266,0.841727,0.302158,0.028777
8,9,3.654505e+09,0.660377,0.226415,0.349057,0.198113,0.952830,0.566038,0.066038,1.981132,0.028302,0.037736,0.018868,0.000000,0.037736,0.603774,0.330189,0.924528,0.273585,0.047170
9,10,4.026463e+09,0.790524,0.718204,0.354115,0.655860,1.887781,0.134663,0.019950,1.209476,0.408978,0.034913,0.189526,0.408978,0.072319,0.730673,0.740648,0.284289,0.229426,0.077307


In [195]:
genre_mean_df=genre_mean_df.drop(columns='user_Timestamp')

In [196]:
genre_mean_df.rename(columns={'user_UserID': 'UserID'}, inplace=True)


In [197]:
final_df = pd.merge(final_merged_data, genre_mean_df, on='UserID', how='left')

In [198]:
final_df.head(70)

,UserID,MovieID,Rating,Timestamp,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,Year,Age,Occupation,Gender_F,Gender_M,user_Action,user_Adventure,user_Animation,user_Children's,user_Comedy,user_Crime,user_Documentary,user_Drama,user_Fantasy,user_Film-Noir,user_Horror,user_Musical,user_Mystery,user_Romance,user_Sci-Fi,user_Thriller,user_War,user_Western
0,1,1193,5,978300760,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1975,1,10,1,0,0.396226,0.377358,1.396226,1.603774,1.094340,0.150943,0.0,1.754717,0.226415,0.000000,0.000000,1.132075,0.000000,0.415094,0.245283,0.207547,0.188679,0.000000
1,1,661,3,978302109,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1996,1,10,1,0,0.396226,0.377358,1.396226,1.603774,1.094340,0.150943,0.0,1.754717,0.226415,0.000000,0.000000,1.132075,0.000000,0.415094,0.245283,0.207547,0.188679,0.000000
2,1,914,3,978301968,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1964,1,10,1,0,0.396226,0.377358,1.396226,1.603774,1.094340,0.150943,0.0,1.754717,0.226415,0.000000,0.000000,1.132075,0.000000,0.415094,0.245283,0.207547,0.188679,0.000000
3,1,3408,4,978300275,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2000,1,10,1,0,0.396226,0.377358,1.396226,1.603774,1.094340,0.150943,0.0,1.754717,0.226415,0.000000,0.000000,1.132075,0.000000,0.415094,0.245283,0.207547,0.188679,0.000000
4,1,2355,5,978824291,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1998,1,10,1,0,0.396226,0.377358,1.396226,1.603774,1.094340,0.150943,0.0,1.754717,0.226415,0.000000,0.000000,1.132075,0.000000,0.415094,0.245283,0.207547,0.188679,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,2,1792,3,978299941,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1998,56,16,0,1,1.519380,0.550388,0.000000,0.000000,0.689922,0.333333,0.0,2.387597,0.023256,0.031008,0.046512,0.000000,0.077519,0.689922,0.472868,0.837209,0.434109,0.100775
66,2,1687,3,978300174,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1997,56,16,0,1,1.519380,0.550388,0.000000,0.000000,0.689922,0.333333,0.0,2.387597,0.023256,0.031008,0.046512,0.000000,0.077519,0.689922,0.472868,0.837209,0.434109,0.100775
67,2,1213,2,978298458,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1990,56,16,0,1,1.519380,0.550388,0.000000,0.000000,0.689922,0.333333,0.0,2.387597,0.023256,0.031008,0.046512,0.000000,0.077519,0.689922,0.472868,0.837209,0.434109,0.100775
68,2,3578,5,978298958,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2000,56,16,0,1,1.519380,0.550388,0.000000,0.000000,0.689922,0.333333,0.0,2.387597,0.023256,0.031008,0.046512,0.000000,0.077519,0.689922,0.472868,0.837209,0.434109,0.100775


In [199]:
final=final_df.drop(columns=['UserID','MovieID','Timestamp'])

In [200]:
final.head()

,Rating,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,Year,Age,Occupation,Gender_F,Gender_M,user_Action,user_Adventure,user_Animation,user_Children's,user_Comedy,user_Crime,user_Documentary,user_Drama,user_Fantasy,user_Film-Noir,user_Horror,user_Musical,user_Mystery,user_Romance,user_Sci-Fi,user_Thriller,user_War,user_Western
0,5,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1975,1,10,1,0,0.396226,0.377358,1.396226,1.603774,1.09434,0.150943,0.0,1.754717,0.226415,0.0,0.0,1.132075,0.0,0.415094,0.245283,0.207547,0.188679,0.0
1,3,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1996,1,10,1,0,0.396226,0.377358,1.396226,1.603774,1.09434,0.150943,0.0,1.754717,0.226415,0.0,0.0,1.132075,0.0,0.415094,0.245283,0.207547,0.188679,0.0
2,3,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1964,1,10,1,0,0.396226,0.377358,1.396226,1.603774,1.09434,0.150943,0.0,1.754717,0.226415,0.0,0.0,1.132075,0.0,0.415094,0.245283,0.207547,0.188679,0.0
3,4,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2000,1,10,1,0,0.396226,0.377358,1.396226,1.603774,1.09434,0.150943,0.0,1.754717,0.226415,0.0,0.0,1.132075,0.0,0.415094,0.245283,0.207547,0.188679,0.0
4,5,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1998,1,10,1,0,0.396226,0.377358,1.396226,1.603774,1.09434,0.150943,0.0,1.754717,0.226415,0.0,0.0,1.132075,0.0,0.415094,0.245283,0.207547,0.188679,0.0


In [103]:
param_dist = {
    'C': np.logspace(-4, 4, 100),   # Regularization strength
    'penalty': ['l1', 'l2']         # Regularization type
}
X=final.drop(columns=['Rating'])
y=final['Rating']
# Create a LogisticRegression instance
clf = LogisticRegression(max_iter=10000)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Perform randomized search with cross-validation
random_search = RandomizedSearchCV(clf, param_distributions=param_dist, n_iter=20, cv=5, scoring='accuracy')
random_search.fit(X_train, y_train)

# Get the best parameters and the best score
best_params = random_search.best_params_
best_score = random_search.best_score_

print(f'Best Parameters: {best_params}')
print(f'Best Score: {best_score:.2f}')

# Use the best model for predictions
best_model = random_search.best_estimator_


C:\Users\20109\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
55 fits failed out of a total of 100.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
55 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\20109\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\20109\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-pac

Best Parameters: {'penalty': 'l2', 'C': 2.782559402207126}
Best Score: 0.38


In [104]:
y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')
print(classification_report(y_test, y_pred))

Accuracy: 0.38
              precision    recall  f1-score   support

           1       0.34      0.04      0.07     11401
           2       0.24      0.02      0.03     21363
           3       0.33      0.30      0.32     52116
           4       0.39      0.72      0.50     69648
           5       0.47      0.21      0.29     45514

    accuracy                           0.38    200042
   macro avg       0.35      0.26      0.24    200042
weighted avg       0.37      0.38      0.33    200042



In [108]:
param_dist_gradient = {
    'n_estimators': [50, 100],       # Number of boosting stages
    'learning_rate': [0.01, 0.1],    # Step size shrinkage
    'max_depth': [3, 4],               # Maximum depth of individual trees
         
}

# Create a GradientBoostingClassifier instance
clf_gradient = GradientBoostingClassifier()

# Perform randomized search with cross-validation
random_search_gradient = RandomizedSearchCV(clf_gradient, param_distributions=param_dist_gradient, n_iter=5, cv=5, scoring='accuracy')
random_search_gradient.fit(X_train, y_train)

# Get the best parameters and the best score
best_params_gradient = random_search_gradient.best_params_
best_score_gradient = random_search_gradient.best_score_

print(f'Best Parameters: {best_params_gradient}')
print(f'Best Score: {best_score_gradient:.2f}')

# Use the best model for predictions
best_model_gradient = random_search_gradient.best_estimator_
y_pred_gradient = best_model_gradient.predict(X_test)

Best Parameters: {'n_estimators': 100, 'max_depth': 4, 'learning_rate': 0.1}
Best Score: 0.40


In [109]:
accuracy = accuracy_score(y_test, y_pred_gradient)
print(f'Accuracy: {accuracy:.2f}')
print(classification_report(y_test, y_pred_gradient))

Accuracy: 0.40
              precision    recall  f1-score   support

           1       0.44      0.14      0.21     11401
           2       0.35      0.02      0.04     21363
           3       0.35      0.30      0.33     52116
           4       0.40      0.73      0.51     69648
           5       0.51      0.25      0.34     45514

    accuracy                           0.40    200042
   macro avg       0.41      0.29      0.28    200042
weighted avg       0.41      0.40      0.36    200042



In [115]:


# Define the hyperparameters and their distributions for random search
param_dist = {
    'n_neighbors': randint(1, 50),
    'weights': ['uniform', 'distance'],
    'algorithm': ['auto']
}
# Create an instance of the KNN model
knn = KNeighborsClassifier()

# Perform random search with cross-validation
randomized_search = RandomizedSearchCV(estimator=knn, param_distributions=param_dist, scoring='accuracy', cv=3, n_iter=5)
randomized_search.fit(X_train, y_train)

# Get the best model and its hyperparameters
best_knn_model = randomized_search.best_estimator_
best_params = randomized_search.best_params_

# Evaluate the model on the test set
accuracy = best_knn_model.score(X_test, y_test)
print("Best Model Accuracy:", accuracy)
print("Best Hyperparameters:", best_params)
# # Saving classifier using pickle
pickle.dump(best_knn_model, open("model_clf_pickle", 'wb'))
  
# load classifier using pickle
# my_model_clf = pickle.load(open("model_clf_pickle", 'rb'))
# result_score = my_model_clf.score(X_test,y_test)
# print("Score: ",result_score)


C:\Users\20109\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\model_selection\_validation.py:824: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\20109\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\model_selection\_validation.py", line 813, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\20109\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_scorer.py", line 266, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
  File "C:\Users\20109\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Py

Best Model Accuracy: 0.35951950090481
Best Hyperparameters: {'algorithm': 'auto', 'n_neighbors': 8, 'weights': 'distance'}


In [201]:
pickle.dump(best_knn_model, open("KNN_model_clf_pickle", 'wb'))

In [203]:
y_predict_knn=best_knn_model.predict(X_test)
accuracy = accuracy_score(y_test, y_predict_knn)
print(f'Accuracy: {accuracy:.2f}')
print(classification_report(y_test, y_predict_knn))

Accuracy: 0.36
              precision    recall  f1-score   support

           1       0.28      0.19      0.23     11401
           2       0.22      0.16      0.19     21363
           3       0.33      0.34      0.33     52116
           4       0.40      0.46      0.43     69648
           5       0.40      0.36      0.38     45514

    accuracy                           0.36    200042
   macro avg       0.32      0.30      0.31    200042
weighted avg       0.35      0.36      0.35    200042



In [208]:
scaler = StandardScaler()

# Fit and transform the scaler on the training data
X_train_scaled = scaler.fit_transform(X_train)

# Transform the test data using the same scaler
X_test_scaled = scaler.transform(X_test)

# Create an SVM classifier instance
clf_SVM= SVC()

# Fit the SVM model on the scaled training data
clf_SVM.fit(X_train_scaled, y_train)

# Make predictions on the scaled test data
y_pred_SVM = clf.predict(X_test_scaled)

# Calculate Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

NotFittedError: This LogisticRegression instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [ ]:
y_pred_SVM = clf_SVM.predict(X_test)



In [ ]:
accuracy = accuracy_score(y_test, y_pred_SVM)
print(f'Accuracy: {accuracy:.2f}')